<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://raw.githubusercontent.com/Arize-ai/phoenix-assets/9e6101d95936f4bd4d390efc9ce646dc6937fb2d/images/socal/github-large-banner-phoenix.jpg" width="1000"/>
        <br>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://arize-ai.slack.com/join/shared_invite/zt-2w57bhem8-hq24MB6u7yE_ZF_ilOYSBw#/shared-invite/email">Community</a>
    </p>
</center>
<h1 align="center">LangChain Tracing</h1>

Let's see how to get started with [LangChain.js](https://js.langchain.com/docs/introduction/) and [OpenInference](https://github.com/Arize-ai/openinference/tree/main/js) to trace your LangChain application using Deno.

> Note: that this example requires the OPENAI_API_KEY environment variable to be set and assumes you are running the Phoenix server on localhost:6006.

In order to run this notebook please run the following command from this directory:
```shell
npm install
```

In [ ]:
import {
  NodeTracerProvider,
  SimpleSpanProcessor,
} from "@opentelemetry/sdk-trace-node";
import { Resource } from "@opentelemetry/resources";
import { OTLPTraceExporter } from "@opentelemetry/exporter-trace-otlp-proto";
import { SEMRESATTRS_PROJECT_NAME } from "@arizeai/openinference-semantic-conventions";

const provider = new NodeTracerProvider({
  resource: new Resource({
    [SEMRESATTRS_PROJECT_NAME]: "deno-langchain",
  }),
});

provider.addSpanProcessor(
  new SimpleSpanProcessor(
    new OTLPTraceExporter({
      url: "http://localhost:6006/v1/traces",
    })
  )
);

provider.register();

console.log("👀 OpenInference initialized");


In [ ]:
import * as lcCallbackManager from "@langchain/core/callbacks/manager";
import { LangChainInstrumentation } from "@arizeai/openinference-instrumentation-langchain";

const lcInstrumentation = new LangChainInstrumentation();
lcInstrumentation.manuallyInstrument(lcCallbackManager);

In [ ]:
import { ChatOpenAI } from "@langchain/openai";
import { ChatPromptTemplate } from "@langchain/core/prompts";

const PROMPT_TEMPLATE = `
    You are a highly trained AI model that can answer any question. Please answer questions concisely. 
    Answer the following question: 
    "{question}"
`;

const question = "What is javascript used for?";

const chatPrompt = ChatPromptTemplate.fromTemplate(PROMPT_TEMPLATE)

const model = new ChatOpenAI({ model: "gpt-4" });

const chain = chatPrompt.pipe(model);

const response = await chain.invoke({ question });

console.log(response.content);
